# Wrangling the Pittsburgh OpenStreetMap Data

Nicholas Cica

nicholasjcica@gmail.com

## Introduction

I will use data munging techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap data for a part of Pittsburgh, PA.  Then, I will use SQL as the data schema to complete my project.

This project will:

* Assess the quality of the data for validity, accuracy, completeness, consistency and uniformity

* Parse and gather data from popular file formats such as .osm and .csv

* Process data from many files and very large files that can be cleaned with spreadsheet programs

* Store, query, and aggregate data using SQL

## Map Area

Pittsburgh, PA
* https://www.openstreetmap.org/relation/188553
* https://mapzen.com/data/metro-extracts/metro/pittsburgh_pennsylvania/

I am originally from Pittsburgh, so I thought it would be fun to revisit my old haunts and see what has changed in the past seven years since I moved away.

## Familiarize Yourself with the Dataset

Unzip the data and examine with:

```
less pittsburgh_pennsylvania.osm
```

Check the file size with:

```
ls -l pittsburgh_pennsylvania.osm
```

Check the open street map documentation

From the wiki, the beginners guide, developers section, map features, osm and eml documentation seem the most useful.  I took some time reading over the documentation before moving on.  
https://wiki.openstreetmap.org/wiki/OSM_XML

### Types of Tags

To understand the OpenStreetMap data, some definitions were necessary:

>A **node** is one of the core elements in the OpenStreetMap data model. It consists of a single point in space defined by its latitude, longitude and node id.
https://wiki.openstreetmap.org/wiki/Node

>A **way** is an ordered list of nodes which normally also has at least one tag or is included within a Relation. Its also a path through a city - a **way** to get from one place to another.
https://wiki.openstreetmap.org/wiki/Way

>A **relation** is one of the core data elements that consists of one or more tags and also an **ordered list** of one or more nodes, ways and/or relations as **members** which is used to define logical or geographic relationships between other elements.
https://wiki.openstreetmap.org/wiki/Relation

## Explore the Data

In [2]:
# All the libararies we will use
import re

import xml.etree.cElementTree as ET
import pprint

from collections import defaultdict

import csv
import codecs
import cerberus
import schema

OSMFILE = "sample6.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
phone_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
state_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

 Counting the tags:

In [3]:
def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename):
        if elem.tag not in tags.keys():
            tags[elem.tag] = 1
        else :
            tags[elem.tag] += 1
    print "Counting the Tags:"
    #pprint.pprint(tags)
    return tags

In [4]:
count_tags(OSMFILE)

Counting the Tags:


{'member': 422,
 'nd': 64150,
 'node': 55216,
 'osm': 1,
 'relation': 76,
 'tag': 36855,
 'way': 5672}

That's a lot of tags!

## Clean the Data

The osm file may be too big to load into memory, so we need to parse the data into smaller bits.  The code will find the way tags and it will return all the sub tags nested within the element for the named parameter. 

### Potential Problems

Before I process the data and add it into the database, I will check the "k" value for each "tag" and see if there are any potential problems.

In [5]:
#"lower", for tags that contain only lowercase letters and are valid,
#"lower_colon", for otherwise valid tags with a colon in their names,
#"problemchars", for tags with problematic characters, and
#"other", for other tags that do not fall into the other three categories.

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        k_val = element.get("k")
        if bool(lower.search(k_val)):
            keys["lower"] += 1
        elif bool(lower_colon.search(k_val)): 
            keys["lower_colon"] += 1
        elif bool(problemchars.search(k_val)):
            keys["problemchars"] += 1
        else:
            keys["other"] += 1
    #print keys
    return keys

def process_keys(filename):
	print "Now let's look for potenatial problems..."
	keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
	for _, element in ET.iterparse(filename):
		keys = key_type(element, keys)
	#pprint.pprint(keys)
	return keys

In [6]:
process_keys(OSMFILE)

Now let's look for potenatial problems...


{'lower': 16617, 'lower_colon': 18994, 'other': 1244, 'problemchars': 0}

* Tags that contain only lowercase letters and are valid: 582,904
* Valid tags with a colon in their names: 664,822
* Tags with problematic characters: 1
* Tags that do not fall into the other three categories: 42,468

### Find out how many unique users have contributed to the map of Pittsburgh

In [7]:
def unique_users(filename):
	print "Let's find out how many unique users have contributed to the map of Pittsburgh..."
	users = set()
	for _, element in ET.iterparse(filename):
		tag = element.tag
		if tag in [ 'node', 'way', 'relation']:
			id = element.attrib['uid']
			users.add(id)
	print 'Number of Unique Users: ', len(users)
	#return users

In [8]:
unique_users(OSMFILE)

Let's find out how many unique users have contributed to the map of Pittsburgh...
Number of Unique Users:  689


### Auditing Street Names, Phone Numbers and State Names

In [9]:
# Streets Names
street_expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Highway", "Way"]

street_mapping = { "St": "Street",
            "St.": "Street", 
            "Av": "Avenue",
            "Av.": "Avenue",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Blvd": "Boulevard",
            "Ct": "Court",
            "Dr": "Drive",
            "Hwy": "Highway",
            "Pl": "Place",
            "Rd": "Road",
            "Rd.": "Road",
            "Sq": "Square",
            }

# Phone Numbers
phone_expected = ('412-', '1-', '724-')

# State Abbr,
state_expected = ["PA"]

state_mapping = {"pa": "PA",
				 "P": "PA",
				 "Pa": "PA",
				 "Ohio": "OH"
				 }

# audit
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    # print 'street', m
    if m:
        street_type = m.group()
        if street_type not in street_expected:
            street_types[street_type].add(street_name)

def audit_phone_number(phone_types, phone_numbers):
	m = phone_type_re.search(phone_numbers)
	# print 'phone', m
	if m:
		phone_type = m.group()
		#if phone_type not in phone_expected:
		if not phone_type.startswith((phone_expected)):
			phone_types[phone_type].add(phone_numbers)

def audit_state_type(state_types, state_name):
    m = state_type_re.search(state_name)
    # print 'state', m
    if m:
        state_type = m.group()
        if state_type not in state_expected:
            state_types[state_type].add(state_name)


# Identify elements
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def is_phone_number(elem):
    return (elem.attrib['k'] == "phone")

# add one for states
def is_state(elem):
    return (elem.attrib['k'] == "addr:state")

# Main Audit Function
def audit(osmfile):
	print "Now its time to audit the data..."
	print "Let's take a look at street names, phone numbers, and state abbrivations..."
	print "\n"
	print "Hmm, something's fishy with our data...some human must have handled the data entry..."
	print "\n"
	osm_file = open(osmfile, "r")
	street_types = defaultdict(set)
	phone_types = defaultdict(set)
	state_types = defaultdict(set)
	for event, elem in ET.iterparse(osm_file, events=("start",)):
		if elem.tag == "node" or elem.tag == "way":
			for tag in elem.iter("tag"):
				if is_street_name(tag):
					audit_street_type(street_types, tag.attrib['v'])
					# testing update - move to shapping
					#better_name = update_street_name(tag.attrib['v'], street_mapping)
					#print 'Updated Street Name: ', better_name
				if is_phone_number(tag):
					# print 'Unformated Phone: ', tag.attrib['v']
					audit_phone_number(phone_types, tag.attrib['v'])
					# testing update - move to shapping
					#formatted_phone_number =  phone_format(tag.attrib['v'])
					#print 'Formated Phone: ', formatted_phone_number
				if is_state(tag):
					#print 'Unformated State: ', tag.attrib['v']
					audit_state_type(state_types, tag.attrib['v'])
					#better_state_name = update_state_name(tag.attrib['v'], state_mapping)
					#print 'Updated State Name: ', better_state_name

	osm_file.close()
	#print 'Street Outliers:', street_types
	#print "\n"
	#print 'Phone Outliers:', phone_types
	#print "\n"
	#print 'State Abbr. Outliers:', state_types
	#print "\n"
	return street_types, phone_types, state_types

Perform the **audit** of street names that don't fit the expected ("Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", "Trail", "Parkway", "Commons", "Circle", "Way", "Broadway", "Colfax")

Perform the **audit** of Phone Numbers that are not in the correct "555-555-5555" formate.

Perform the **audit** of States which are not abbriviated correctly to PA.

In [10]:
audit(OSMFILE)

Now its time to audit the data...
Let's take a look at street names, phone numbers, and state abbrivations...


Hmm, something's fishy with our data...some human must have handled the data entry...




(defaultdict(set,
             {'30': {'State Route 30'},
              '51': {'Route 51'},
              '519': {'PA 519'},
              '885': {'Route 885'},
              'Alley': {'Pine Alley'},
              'Allies': {'Boulevard of the Allies'},
              'Ave': {'Centre Ave', 'Fifth Ave', 'S Negley Ave'},
              'Blvd': {'Washington Blvd'},
              'Circle': {'Bardona Circle',
               'Chardonnay Circle',
               'Golden Circle',
               'Jefferson Pointe Circle',
               'Laurel Ridge Circle',
               'Longview Circle',
               'Marion Circle',
               'Morning Wind Circle',
               'Naughton Circle',
               'Oakhurst Circle',
               'Redrome Circle',
               'Steeplechase Circle',
               'Trotwood Circle',
               'Wheatland Circle',
               'Winners Circle'},
              'Dr': {'Douglas Dr', 'Glengary Dr', 'Kirkwall Dr', 'Selvin Dr'},
              'East': 

## Preparing for Database - SQL

After auditing is complete the next step is to prepare the data to be inserted into a SQL database.
To do so I will clean and parse the elements in the OSM XML file, transforming them from document format to
tabular format, thus making it possible to write to .csv files.  These csv files can then easily be
imported to a SQL database as tables.

The process for this transformation is as follows:
- Use iterparse to iteratively step through each top level element in the XML
- Shape each element into several data structures using a custom function
- Utilize a schema and validation library to ensure the transformed data is in the correct format
- Write each data structure to the appropriate .csv files

In [11]:
#print "Now we have to clean up their mess!"
def update_street_name(name, mapping):
    m = street_type_re.search(name)
    better_name = name
    if m:
        if m.group() in mapping.keys():
            #print 'BEFORE'
            #print name
            better_name = re.sub(m.group(),mapping[m.group()], name)
            #print 'AFTER'
            #print better_name
        # else:
            # print 'Not in Mapping:', m.group()
    return better_name

def phone_format(phone_number):
    clean_phone_number = re.sub('[^0-9]+', '', phone_number)
    formatted_phone_number = re.sub("(\d)(?=(\d{3})+(?!\d))", r"\1-", "%d" % int(clean_phone_number[:-1])) + clean_phone_number[-1]
    return formatted_phone_number

def update_state_name(name, mapping):
	m = state_type_re.search(name)
	better_name = name
	if m:
		if m.group() in mapping.keys():
			#print 'BEFORE'
			#print name
			better_name = re.sub(m.group(),mapping[m.group()], name)
			#print 'AFTER'
			#print better_name
		# else:
			# print 'Not in Mapping:', m.group()
	return better_name


# SHAPE
OSM_PATH = "sample6.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
					problem_chars=PROBLEMCHARS, default_tag_type='regular'):
	"""Clean and shape node or way XML element to Python dict"""

	node_attribs = {}
	way_attribs = {}
	way_nodes = []
	tags = [] 

	if element.tag == 'node':
		for node in NODE_FIELDS:
			node_attribs[node] = element.attrib[node]
		for child in element:
			tag = {}
			if PROBLEMCHARS.search(child.attrib["k"]):
				continue
	
			elif LOWER_COLON.search(child.attrib["k"]):
			
				tag_type = child.attrib["k"].split(':',1)[0]
				tag_key = child.attrib["k"].split(':',1)[1]
				tag["key"] = tag_key
				if tag_type:
					tag["type"] = tag_type
				else:
					tag["type"] = 'unspecified'
			
				tag["id"] = element.attrib["id"]
				if child.attrib['k'] == "addr:street":
					tag['value'] = update_street_name(child.attrib['v'], street_mapping)
					#print "Node (problem) address updated"
				if child.attrib['k'] == "phone":
					tag['value'] = phone_format(child.attrib['v'])
					#print "Node (problem) phone updated"
				if child.attrib['k'] == "addr:state":
					tag['value'] = update_state_name(child.attrib['v'], state_mapping)
					#print "Node (problem) state updated"
				else:
					tag["value"] = child.attrib["v"]
			else:
				if child.attrib['k'] == "addr:street":
					tag['value'] = update_street_name(child.attrib['v'], street_mapping)
					#print "Node address updated"
				if child.attrib['k'] == "phone":
					tag['value'] = phone_format(child.attrib['v'])
					#print "Node phone updated"
				if child.attrib['k'] == "addr:state":
					tag['value'] = update_state_name(child.attrib['v'], state_mapping)
					#print "Node state updated"
				else:
					tag["value"] = child.attrib["v"]
				tag["key"] = child.attrib["k"]
				tag["type"] = "unspecified"
				tag["id"] = element.attrib["id"]
			if tag:
				tags.append(tag)
			return {'node': node_attribs, 'node_tags': tags}
	
	elif element.tag == 'way':
		for way in WAY_FIELDS:
			way_attribs[way] = element.attrib[way]
		for child in element:
			nd = {}
			tag = {}
			if child.tag == 'tag':
				if PROBLEMCHARS.search(child.attrib["k"]):
					continue
				elif LOWER_COLON.search(child.attrib["k"]):
					tag_type = child.attrib["k"].split(':',1)[0]
					tag_key = child.attrib["k"].split(':',1)[1]
					tag["key"] = tag_key
					if tag_type:
						tag["type"] = tag_type
					else:
						tag["type"] = 'unspecified'
					tag["id"] = element.attrib["id"]
					if child.attrib['k'] == "addr:street":
						tag['value'] = update_street_name(child.attrib['v'], street_mapping)
						#print "Way (problem) address updated"
					if child.attrib['k'] == "phone":
						tag['value'] = phone_format(child.attrib['v'])
						#print "Way (problem) phone updated"
					if child.attrib['k'] == "addr:state":
						tag['value'] = update_state_name(child.attrib['v'], state_mapping)
						#print "Way (problem) state updated"
					else:
						tag["value"] = child.attrib["v"]
	
				else:
					if child.attrib['k'] == "addr:street":
						tag['value'] = update_street_name(child.attrib['v'], street_mapping)
						#print "Way address updated"
					if child.attrib['k'] == "phone":
						tag['value'] = phone_format(child.attrib['v'])
						#print "Way phone updated"
					if child.attrib['k'] == "addr:state":
						tag['value'] = update_state_name(child.attrib['v'], state_mapping)
						#print "Way state updated"
					else:
						tag["value"] = child.attrib["v"]
					tag["key"] = child.attrib["k"]
					tag["type"] = "unspecified"
					tag["id"] = element.attrib["id"]
				if tag:
					tags.append(tag)

			elif child.tag == 'nd':
				nd['id'] = element.attrib["id"]
				nd['node_id'] = child.attrib["ref"]
				nd['position'] = len(way_nodes)
	
				if nd:
					way_nodes.append(nd)
			else:
				continue
		return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""
    print "Now Let's clean and prepare the data for our database!"

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

 	print "Data processing complete!"

Ok!  let's process the data Map of Pittsburgh!

In [12]:
process_map(OSM_PATH, validate=True)

Now Let's clean and prepare the data for our database!
Data processing complete!


This process has split the osm file into five cvs files:

* nodes_tags.csv
* nodes.csv
* ways.csv
* ways_tags.csv
* ways_nodes.csv

Now we need to set up the database and get those files into it!

## Import the cleaned .csv files into a SQL database

```
sqlite3 pittsburgh.db
```

### Create the Tables (Schema)

```
CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);

CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id)
);

CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT
);

CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);

CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);
```

### Verify the Tables were Created

```
.tables
```

### Import the Data

```
.mode csv
.import nodes_tags.csv nodes_tags
.import nodes.csv nodes
.import ways.csv ways
.import ways_tags.csv ways_tags
.import ways_nodes.csv ways_nodes
```

Sweet!  Now we can run all kinds of crazy queries against the database and find out cool things about Pittsburgh.

# Overview of the Data

This section contains basic statistics about the dataset, the SQL queries used to gather them, the DB API python code, and some additional ideas about the data in context.

### File Sizes

* pittsburgh_pennsylvania.osm: 431.6 MB
* pittsburgh.db: 237.6 MB
* nodes.csv: 162 MB
* nodes_tags.csv: 9.8 MB
* ways.csv: 11.9 MB
* ways_tags.csv: 34.2 MB
* ways_nodes.csv: 53.1 MB

### Number of Nodes

In [52]:
import sqlite3

db = sqlite3.connect("pittsburgh.db")
c = db.cursor()
query = "SELECT COUNT(*) FROM nodes;"
c.execute(query)
rows = c.fetchall()
db.close()

print "Number of Nodes: ", rows[0][0]

Number of Nodes:  1932541


### Number of Ways

In [53]:
db = sqlite3.connect("pittsburgh.db")
c = db.cursor()
query = "SELECT COUNT(*) FROM ways;"
c.execute(query)
rows = c.fetchall()
db.close()

print "Number of Ways: ", rows[0][0]

Number of Ways:  198508


### Number of Unique Users

In [56]:
db = sqlite3.connect("pittsburgh.db")
c = db.cursor()
query = "SELECT COUNT(DISTINCT(u.uid)) \
FROM (SELECT uid \
      FROM nodes \
      UNION ALL \
      SELECT uid \
      FROM ways) u;" 
c.execute(query)
rows = c.fetchall()
db.close()

print "Number of Unique Users: ", rows[0][0]

Number of Unique Users:  1253


### Top 10 Contributing Users

In [59]:
db = sqlite3.connect("pittsburgh.db")
c = db.cursor()
query = "SELECT u.user, COUNT(*) as num \
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) u \
GROUP BY u.user \
ORDER BY num DESC \
LIMIT 10;" 
c.execute(query)
rows = c.fetchall()
db.close()

print "Top 10 Contributing Users: ", rows

Top 10 Contributing Users:  [(u'GeoKitten', 261073), (u'woodpeck_fixbot', 204981), (u'rickmastfan67', 128972), (u'behemoth14', 101086), (u'Gary Hayden', 95026), (u'abbafei', 93662), (u'Fredlyfish4', 80402), (u'dchiles', 72425), (u'AndrewSnow', 72365), (u'TIGERcnl', 63198)]


### Number of Users Appearing Only Once (having 1 post)

In [61]:
db = sqlite3.connect("pittsburgh.db")
c = db.cursor()
query = "SELECT COUNT(*) FROM (SELECT u.user, COUNT(*) as num \
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) u \
GROUP BY u.user \
HAVING num = 1);" 
c.execute(query)
rows = c.fetchall()
db.close()

print "Number of Users Appearing Only Once: ", rows[0][0]

Number of Users Appearing Only Once:  181


### List of Top 20 Amenities in Pittsburgh

In [71]:
db = sqlite3.connect("pittsburgh.db")
c = db.cursor()
query = "SELECT value, COUNT(*) as num \
FROM nodes_tags \
WHERE key = 'amenity' \
GROUP BY value \
ORDER BY num DESC \
LIMIT 20;" 
c.execute(query)
rows = c.fetchall()
db.close()

print "List of Top 20 Amenities in Pittsburgh: ", rows

List of Top 20 Amenities in Pittsburgh:  [(u'place_of_worship', 1444), (u'school', 1209), (u'restaurant', 660), (u'grave_yard', 374), (u'fast_food', 291), (u'library', 226), (u'post_office', 200), (u'bench', 191), (u'fuel', 188), (u'waste_basket', 175), (u'parking', 171), (u'bank', 143), (u'post_box', 134), (u'police', 126), (u'cafe', 100), (u'kindergarten', 95), (u'pharmacy', 92), (u'fire_station', 85), (u'bar', 75), (u'toilets', 72)]


### How many Starbucks are there?

In [77]:
db = sqlite3.connect("pittsburgh.db")
c = db.cursor()
query = "SELECT value, COUNT(*) as num \
FROM nodes_tags \
WHERE key = 'name' \
GROUP BY value \
ORDER BY num DESC \
LIMIT 20;" 
c.execute(query)
rows = c.fetchall()
db.close()

print rows

[(u'Subway', 49), (u"McDonald's", 43), (u'Giant Eagle', 33), (u'Rite Aid', 28), (u'PNC Bank', 26), (u'Starbucks', 25), (u"Wendy's", 24), (u'GetGo', 23), (u'Hydrant', 21), (u'Sheetz', 21), (u'Pizza Hut', 19), (u'Burger King', 18), (u"Arby's", 16), (u'Sunoco', 16), (u'Dairy Queen', 15), (u'Dollar General', 14), (u'First Baptist Church', 12), (u'H&R Block', 12), (u'Saint Johns Church', 12), (u'CVS', 11)]


### List of Top 5 Cuisine in Pittsburgh

In [66]:
db = sqlite3.connect("pittsburgh.db")
c = db.cursor()
query = "SELECT nodes_tags.value, COUNT(*) as num \
FROM nodes_tags \
JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value = 'restaurant') r ON nodes_tags.id = r.id \
WHERE nodes_tags.key = 'cuisine' \
GROUP BY nodes_tags.value \
ORDER BY num DESC \
LIMIT 5;" 
c.execute(query)
rows = c.fetchall()
db.close()

print "List of Top 5 Cuisine in Pittsburgh: ", rows

List of Top 5 Cuisine in Pittsburgh:  [(u'american', 131), (u'pizza', 77), (u'italian', 38), (u'chinese', 30), (u'mexican', 20)]


### Religions at a Glance

In [80]:
db = sqlite3.connect("pittsburgh.db")
c = db.cursor()
query = "SELECT nodes_tags.value, COUNT(*) as num \
FROM nodes_tags \
JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship') r ON nodes_tags.id = r.id \
WHERE nodes_tags.key='religion' \
GROUP BY nodes_tags.value \
ORDER BY num DESC \
LIMIT 5;" 
c.execute(query)
rows = c.fetchall()
db.close()

print "Religions at a Glance: ", rows

Religions at a Glance:  [(u'christian', 1360), (u'jewish', 14), (u'muslim', 3), (u'unitarian_universalist', 2), (u'eckankar', 1)]


Not surprising that Christianity is well represented. Pittsburgh has some of the most beautfil churches that also brew beer.

Apparently, **Eckankar** (meaning Co-worker with God) is a modern-day religion where followers believe its purpose is to help individuals find their way back to God through direct personal spiritual experiences.  Huh, learn something everyday.

## Suggestions for Improving and Analyzing the Data

I think the next step would be to visualize the data and look for relationships.  If we were really bold, we could use this data from one American city and use machine learning to predict outcomes about other, similarly sized American cities and maybe even be able to make predictions based on population size.

### Benefits and Anticipated Problems in Implementing the Improvement

One benefit of predicting city data from other city data is measuring the effectives of public works.  We may be able to utilize a prediction based on data by measuring the number of police stations, fire stations, hospitals, etc. against the crime rate, emergency first response time, and level of health care. Since our data was entered by hand (in this case), we need to be careful if our predicted data doesn’t match the actual data. We need to use other metrics to measure the validity of the data to be sure that the predictions are within an accepted range.

The problem we could face in implimenting such an analysis is that:
1. we need the data
2. a pattern must exist

Without those two things, we would not be able to learn from data.

# Conclusion

There is definietly something to be said by the amount of human error that goes into a dataset such as this.  Only a domain expert of the area would be able to - at a glance - clean the data on a case by case basis.  That's why using these data analysis tools are important.  It allows us to explore the data and look for outliers and then clean the data without having to know the ins and outs of all the data we are sifting through.

# References

Shout out to the DataScience slack group for helping me figure out how to import the data into my database!
https://udacitydatascience.slack.com/messages/p3-openstreetmap/

Markdown help: https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#code

Wikipedia for fact checking this Eckankar religion: https://en.wikipedia.org/wiki/Eckankar